In [7]:
import tika
from tika import parser
#создает и запускает каждый раз /tmp/tika-server.jar
import striprtf
from striprtf.striprtf import rtf_to_text
import aspose.words as aw
import docx          
from docx import Document
import docx2txt  
from pdf2image import convert_from_path
from guess_language import guess_language
import langid
import cld3

In [8]:
from PIL import Image
import cv2 
import pytesseract
from pytesseract import Output

In [9]:
import os
import re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

Считываение текстов из загруженных документов. Для выбора представляется два парсера tika и aspose.words. Оба читают все типы документов. По умолчанию aspose.

In [10]:
def read_train(file_list, path, pars="aw"):
    '''Считывание документов из папки для добавления в датафрейм.
    На входе:
       file_list - список файлов
       path - папка, в которой лежат файлы
       pars - парсер. Варианты: tika(все форматы), aspose.words(все), textract(все, особый способ для rtf),
                                docx(только docx), rtf(rtf), pdf(pdf).
           Проверка на корректность pdf, если текст не распознается как русский - распознавание pytesseract,
           если текст отсутсвет, по умолчанию "".
    На выходе:
       texts - список той же длины, что и file_list'''
    
    texts = []   #все тексты договоров
    for i in tqdm(file_list[0:]):
        filename = path+i
        if os.path.exists(filename):
            #проверку, чтобы docx тотолько в docx
            if pars=="tika":
                parsed = parser.from_file(filename)
                #text = parsed["content"]
                text = easy_clean(parsed["content"], parsed["metadata"])                
            elif pars=="aw":
                parsed = aw.Document(filename).get_text()
                text = aw_clean(parsed)

            if filename.split('.')[-1] == "pdf":
                #у pdf проверяем адекватность текста, если мусор - распознаем изображениями
                if guess_language(text) != "ru":
                    text = ''
                    pages = convert_from_path(filename, 500)
                    for i, page in enumerate(pages):
                        parsed = pytesseract.image_to_string(page, lang="rus")
                        text += parsed
                    text = easy_clean(text, [])
                else:
                    pass
                                                    
            texts.append(text)
        else:
            texts.append("")
            print(filename, ' не существует')
    return texts


Простая очитска текста от мусора и лишних пробелов.

In [15]:
def aw_clean(s):
    #простая очистка текста
    replace_dict={"Скачано с":" ", 'Образец документа':' ', 
                 'подготовлен сайтом\x13 HYPERLINK "https://dogovor-obrazets.ru"':' ',
                 '\x14https://dogovor-obrazets.ru\x15':' ',
                 'Источник страницы с документом:\x13 HYPERLINK "https://dogovor-obrazets.ru':' ',
                 '/Ð´Ð¾Ð³Ð¾Ð²Ð¾Ñ\x80/Ð\x9eÐ±Ñ\x80Ð°Ð·ÐµÑ\x86_Ð\x94Ð¾Ð³Ð¾Ð²Ð¾Ñ\x80_Ð¿Ð¾Ñ\x81Ñ':' ',
                  '\x82Ð°Ð²ÐºÐ¸_Ñ\x82Ð¾Ð²Ð°Ñ\x80Ð°-1" \x14https://dogovor-obrazets.ru/договор/':' ',
                  "\* MERGEFORMAT": " ", "FILLIN": " ",
                 'Evaluation Only. Created with Aspose.Words. Copyright 2003-2023 Aspose Pty Ltd.': " ",
                 'Created with an evaluation copy of Aspose.Words.': " ",
                 'To discover the full versions of our APIs please visit: https://products.aspose.com/words/':" ",
                 "This document was truncated here because it was created in the Evaluation Mode.": " ", 
                 }
    for key, value in replace_dict.items():
        s = s.replace(key, value)
    s = re.sub(r"https?://[^,\s]+,?", " ", s) #удаление гиперссылок
    s = re.sub('"consultantplus://offline/ref=[0-9A-Z]*"', '', s)
    s = re.sub(r'HYPERLINK \\l "Par[0-9]*"', '', s)
    s = re.sub(r'HYPERLINK', '', s)
    #re.sub(r'\(<^\)>+\)', '', string) удалить <1>
    #s = re.sub(r"^\s+|\n|\t|\r|\s+$", " ", s) #удаление пробелов в начале и конце строки, переносов
    replace_dict={" .": ".", " ,": ",", " « »": "", "« » ": "", " « »": "",' " "': "", " “ ”": "",  
                  "_":" ", "�":" ", "·": "", "--":"", "…":"", "/":"", "|":"", '“”':'', "®": " ", "\d": " ", 
                  '\x0e':" ", "\x02":"", "\x0c":" ", "\x07":" ", "\xa0":" ", "\x13":" ", "\x14":" ", "\x15":" "} 
    for key, value in replace_dict.items():
        s = s.replace(key, value) 
    s = re.sub(r"\s+", " ", s).strip() #удаление пробелов в начале и конце строки, переносов \r\n\t
    replace_dict={" « »":"", " “ ”":"", "( ) ": ""}
    for key, value in replace_dict.items():
        s = s.replace(key, value) 
    #s = s.capitalize()
    return s

def easy_clean(s, t):
    #простая очистка текста
    try:
        s = s.replace(t['title'], "") #parsed["content"]['title']
    except:
        pass
    replace_dict={"Скачано с":" ", 'Образец документа':'', 
                 'подготовлен сайтомhttps://dogovor-\nobrazets.ru':'',
                 'Источник страницы с документом:https://dogovor-\nobrazets.ru/договор/':'',
                 }
    for key, value in replace_dict.items():
        s = s.replace(key, value)
    s = re.sub(r'convert.*filter : Text', ' ', s) #при открытии doc через txt
    #s = re.sub(r'Выше приводится.*особенностей и условий.', ' ', s)
    s = re.sub(r"https?://[^,\s]+,?", " ", s) #удаление гиперссылок
    #s = re.sub(r"^\s+|\n|\t|\r|\s+$", " ", s) #удаление пробелов в начале и конце строки, переносов
    replace_dict={" .": ".", " ,": ",", " « »": "", "« » ": "", " « »": "",' " "': "", " “ ”": "", "• ": " ", 
                  "_":" ", "�":" ", "·": "", "--":"", "…":"", "/":"", "|":"", '“”':'', "\d": " ", "\ufeff":" ",
                   '\x0e':" ", "\x02":"", "\x0c":" ", "\x07":" ", "\xa0":" ", "\x13":" ", "\x14":" ", "\x15":" "} 
    for key, value in replace_dict.items():
        s = s.replace(key, value)
    s = re.sub(r"\s+", " ", s).strip() #удаление пробелов в начале и конце строки, переносов
    replace_dict={" « »":"", " “ ”":"", "( ) ": ""}
    for key, value in replace_dict.items():
        s = s.replace(key, value)
    #s = s.capitalize()
    return s

Создаем таблицу со списком документов и их типами. Проходим по всем папкам в целевой директории и записываем названия файлов в одну таблицу.

In [12]:
dataPath = r"dataset/"
docDir = {r"Договоры/": 'contract', r"Оферты/": 'contract offer', r"Заявления/": 'application', 
          r"Приказы/": 'order', r"Соглашения/": 'arrangement', r"Уставы/": 'statute', r"Решения/": 'determination'}

data = pd.DataFrame(columns=["url", 'class'])
for dir_ in  docDir:
    #print(docDir[dir_])
    dirPath = dataPath + dir_
    result = [dir_ + f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]
    #data.append(pd.DataFrame(data={"url":result, "class":docDir[dir_]}))
    data = pd.concat([data, pd.DataFrame(data={"url":result, "class":docDir[dir_]})], ignore_index=True)

In [13]:
data

,url,class
0,Договоры/dogovor-scheta-tsifrovogo-rublia-mezh...,contract
1,Договоры/dogovor-mezhdu-rossiiskoi-federatsiei...,contract
2,Договоры/242b138fd2025b7a5207876098355e03.docx,contract
3,Договоры/eddc39f5f675890f285014c886bf22c9.doc,contract
4,Договоры/tipovoi-dogovor-kupli-prodazhi-zemeln...,contract
...,...,...
852,Решения/Решение Коллегии Евразийской экономиче...,determination
853,Решения/Решение Совета Евразийской экономическ...,determination
854,Решения/reshenie-_-634-ot-28.03.2024.pdf,determination
855,Решения/Решение Евразийского межправительствен...,determination


Считываем тексты документов по созданному ранее списку и объединяем их в одну таблицу

In [16]:
texts = read_train(file_list=list(data.url), path='dataset/', pars="aw")
data['text']=texts

  0%|          | 0/857 [00:00<?, ?it/s]

In [17]:
data

,url,class,text
0,Договоры/dogovor-scheta-tsifrovogo-rublia-mezh...,contract,"Приложение к письму Банка России ""О форме дого..."
1,Договоры/dogovor-mezhdu-rossiiskoi-federatsiei...,contract,ДОГОВОР МЕЖДУ РОССИЙСКОЙ ФЕДЕРАЦИЕЙ И РЕСПУБЛИ...
2,Договоры/242b138fd2025b7a5207876098355e03.docx,contract,"Договор {ДатаДокумента} г. {НазваниеКонтр}, в ..."
3,Договоры/eddc39f5f675890f285014c886bf22c9.doc,contract,"ДОГОВОР г. Москва 2020 года Гр. РФ , года рожд..."
4,Договоры/tipovoi-dogovor-kupli-prodazhi-zemeln...,contract,Зарегистрировано в Минюсте РФ 25 августа 1992 ...
...,...,...,...
852,Решения/Решение Коллегии Евразийской экономиче...,determination,КОЛЛЕГИЯ ЕВРАЗИЙСКОЙ ЭКОНОМИЧЕСКОЙ КОМИССИИ РЕ...
853,Решения/Решение Совета Евразийской экономическ...,determination,СОВЕТ ЕВРАЗИЙСКОЙ ЭКОНОМИЧЕСКОЙ КОМИССИИ РЕШЕН...
854,Решения/reshenie-_-634-ot-28.03.2024.pdf,determination,Российская Федерация Курганская область Муници...
855,Решения/Решение Евразийского межправительствен...,determination,ЕВРАЗИЙСКИЙ МЕЖПРАВИТЕЛЬСТВЕННЫЙ СОВЕТ РЕШЕНИЕ...


Загружаем документы из csv таблицы

In [18]:
sample = pd.read_csv('dataset/sample.csv', sep=',', index_col=None)
sample.head()

,class,text
0,arrangement,СОГЛАШЕНИЕ N 8\nо расторжении трудового догово...
1,arrangement,Соглашение о предоставлении опциона на заключе...
2,arrangement,Соглашение\nо реструктуризации задолженности\n...
3,arrangement,Дополнительное соглашение\r\nк договору купли-...
4,arrangement,Соглашение\nо расторжении договора об оказании...


Общий маппинг классов:  

proxy - доверенность  
contract - договор  
act - акт  
application - заявление  
order - приказ  
invoice - счет  
bill - приложение  
arrangement - соглашение  
contract offer - договор оферты  
statute - устав  
determination - решение  

In [20]:
sample['class'].value_counts()

class
proxy             71
contract          70
act               69
application       61
order             50
invoice           43
bill              41
arrangement       40
contract offer    25
statute           21
determination     10
Name: count, dtype: int64

Объединяем все тексты документов в один файл

In [23]:
train = pd.concat([data[['class', 'text']], sample[['class', 'text']]], ignore_index=True)
train

,class,text
0,contract,"Приложение к письму Банка России ""О форме дого..."
1,contract,ДОГОВОР МЕЖДУ РОССИЙСКОЙ ФЕДЕРАЦИЕЙ И РЕСПУБЛИ...
2,contract,"Договор {ДатаДокумента} г. {НазваниеКонтр}, в ..."
3,contract,"ДОГОВОР г. Москва 2020 года Гр. РФ , года рожд..."
4,contract,Зарегистрировано в Минюсте РФ 25 августа 1992 ...
...,...,...
1353,bill,Счет № 5 от 01 октября 2020 г.\r\n\r\nПоставщи...
1354,bill,Счет на оплату № от 14 октября 2020 года\r\n\r...
1355,bill,Счет №23 от 12.09.2024 г.\t\t...
1356,bill,"""Огурец!"" (ИП Микрюков В.В.)\t\t\t\t\t\t\r\n\t..."


Сохраняем датасет с текстами и названиями классов для дальнейшей работы

In [24]:
train.to_csv('dataset/train.csv', index=False)